# Data, Correlation, & Fitting Data

## Bivariate Data

As the name implies, bivariate data is data that consists of two variables. We compare and analyze the two variables with respect to one another in order to attempt to find and/or explain the relationship between them. It is also possible that one of these variables depends on the other, in which case we have an independent variable and a dependent variable.

* altitude and air density
* ice cream sales and temperature throughout a day
* mana cost of a MTG card and the turns remaining in a game

When we have bivariate data, there are some simple things we can do to help us understand what sort of relationship our variables have with one another. We want to be able to visualize our data as well as visualize the relationship (if there is one) and quantify it. We can use a combination of `pandas`, `numpy`, and `matplotlib`/`seaborn`/`bokeh` to handle the analysis and visualization.

### Simple Example

First let's consider data from the *U.S. Standard Atmostphere 1976*. We simply want to look at how the density of air changes as we move higher into the atmosphere. In our simple data set (a full one has been provided) consists of altitudes in meters and densities in kilograms per meters cubed.

The original data and model can be found in its original form, provided by NASA [here](https://ntrs.nasa.gov/api/citations/19770009539/downloads/19770009539.pdf). We however are using a stripped down version sourced from [here](https://www.engineeringtoolbox.com/standard-atmosphere-d_604.html).

In [ ]:
import pandas as pd
atmos = pd.read_csv('data/atmosphere_simple.csv')
atmos

In [ ]:
import hvplot.pandas
# general note on hvPlot vs Holoviews - usually we will create a plot using hvPlot when we have a dataframe,
# and then use Holoviews to manipulate and compose it.

(atmos.hvplot.line(x='altitude', y='air_density') * atmos.hvplot.scatter(x='altitude', y='air_density')).opts(
    xlabel='Altitude (m)',
    ylabel='Density of Air ($kg/m^{3}$)',
    title='U.S. Standard Atmosphere, Altitude vs. Density of Air',
    xlim=(-5000, 85000),
    height=480,
    show_grid=True
)

We can see that there is a pretty strong relationship between these two variables (in the direction that we expect - at higher altitudes air is much thinner!). We can see too that this relationship is non-linear - air density rapidly decreases as we ascend to about 20km in altitude. It seems to asymtotically approach a density of 0 as we continue upward (in reality it is not asymtotic, as it is around 100000m (100km), that atmospheric pressue, and thus the density of air, becomes 0).

You may ask why did we plot altitude on the x-axis, when it makes more sense to represent it vertically? The answer is actually very simple: altitude in an indepenent variable in this context! The graph above tells us that as the altitude increases, the density of air decreases. Let's flip it and see what it loops like: 

In [ ]:
(atmos.hvplot.line(x='air_density', y='altitude') * atmos.hvplot.scatter(x='air_density', y='altitude')).opts(
    xlabel='Density of Air ($kg/m^{3}$)',
    ylabel='Altitude (m)',
    title='U.S. Standard Atmosphere, Density of Air vs. Altitude',
    xlim=(-0.1, 1.4),
    height=480,
    show_grid=True
)

This swapped ordering is still technically valid as it still shows how the data is related, but now indicates that the changes in alitude are caused by changes in air density (which is not true!). We need to take care when selecting what data goes on specific axes.

## Multivariate Data

As the name implies, multivariate data is data that consists of more than two variables. The idea is the same as it is with bivariate data, except we have more variables to work with.

### Simple Example

Again let's consider data from the *U.S. Standard Atmostphere 1976*.

The original data and model can be found in its original form, provided by NASA [here](https://ntrs.nasa.gov/api/citations/19770009539/downloads/19770009539.pdf). We however are using a stripped down version sourced from [here](https://www.engineeringtoolbox.com/standard-atmosphere-d_604.html).

In [ ]:
import pandas as pd
atmos = pd.read_csv('data/atmosphere.csv')
atmos

We can look at any two variables with respect to altitude (our independent variable!):

In [ ]:
import hvplot.pandas

# it can get annoying to repeat a lot of common parameters when creating overlays; we can put such common
# parameters in a dictionary, and then expand that dictionary whenever we want to use those parameters in
# a function call
plot_axes = {
    'x': 'altitude',
    'y': 'atmospheric_pressure',
}

altitude_vs_pressure = atmos.hvplot.line(**plot_axes) * atmos.hvplot.scatter(**plot_axes)

altitude_vs_pressure.opts(
    xlabel='Altitude (m)',
    ylabel='Atmospheric Pressure ($$10^{-5}N s/m^{2}$$)',
    title='Altitude vs. Atmospheric Pressure',
    xlim=(-5000, 85000),
    height=480,
    show_grid=True
)

In [ ]:
plot_axes = {
    'x': 'altitude',
    'y': 'temperature',
}

altitude_vs_temperature = atmos.hvplot.line(**plot_axes) * atmos.hvplot.scatter(**plot_axes)

altitude_vs_temperature.opts(
    xlabel='Altitude (m)',
    ylabel='Temeprature ($^{\circ}C$)',
    title='Altitude vs. Temperature',
    xlim=(-5000, 85000),
    height=480,
    show_grid=True    
)

Or we can plot them simulataneously on the same figure

In [ ]:
plot_axes = {
    'x': 'altitude',
    'y': ['atmospheric_pressure', 'temperature'],
}

altitude_vs_pressure_and_temperature = atmos.hvplot.line(**plot_axes) * atmos.hvplot.scatter(**plot_axes)

altitude_vs_pressure_and_temperature.opts(
    xlabel='Altitude (m)',
    title='Altitude vs. Atmospheric Pressure & Temperature',
    xlim=(-5000, 85000),
    height=480,
    show_grid=True    
)

The problem with setting both of these variables on the same plot is that numerically they are within very different ranges of values. Temperature fluctuates from 20 down to nearly -80, and our density only ranges from a little over 10.0 to nearly 0.0.

There are different ways to handle this, but ideally we can just use a separate y-axis on the right side of our plot. There is only light support of this outside of `matplotlib`, but we can make it work. THe biggest problem is that we need to compose the plots slightly more granuarly to ensure that enabling the twin-axis works properly.

In [ ]:
# compose the first plot
plot_axes = {
    'x': 'altitude',
    'y': 'atmospheric_pressure',
}
altitude_vs_pressure = atmos.hvplot.line(**plot_axes) * atmos.hvplot.scatter(**plot_axes)

# compose the second plot
plot_axes = {
    'x': 'altitude',
    'y': 'temperature',
}
altitude_vs_temperature = atmos.hvplot.line(**plot_axes) * atmos.hvplot.scatter(**plot_axes)

# compoose everything together, and enable the twin-axis using the `multi_y` parameter
(altitude_vs_temperature * altitude_vs_pressure).opts(
    xlabel='Altitude (m)',
    title='Altitude vs. Atmospheric Pressure & Temperature',
    xlim=(-5000, 85000),
    height=480,
    show_grid=True,
    multi_y=True
)

Now we can see both data sets in better detail (specifically atmospheric pressure), as temperature is added to a secondary axis. This allows us to avoid visually squashing the pressure data. Throwing more data into this plot however would not work well, as we cannot add any more axes, and everything at varying scales becomes unmaintainable. A this point we would defer to composing multiple plots in a typcal layout without any overlays. We could also relay on using colors and size (as we have seen before) for representing multiple variables.

In [ ]:
import holoviews as hv

# compose the first plot, 
plot_axes = {
    'x': 'altitude',
    'y': 'atmospheric_pressure'
}
altitude_vs_pressure = atmos.hvplot.line(**plot_axes) * atmos.hvplot.scatter(**plot_axes, s='dynamic_viscosity').opts(size=hv.dim('dynamic_viscosity')**4)

# compose the second plot
plot_axes = {
    'x': 'altitude',
    'y': 'temperature',
}
altitude_vs_temperature = atmos.hvplot.line(**plot_axes) * atmos.hvplot.scatter(**plot_axes, s='dynamic_viscosity').opts(size=hv.dim('dynamic_viscosity')**4)

# compoose everything together, and enable the twin-axis using the `multi_y` parameter
(altitude_vs_temperature * altitude_vs_pressure).opts(
    xlabel='Altitude (m)',
    title='Altitude vs. Atmospheric Pressure & Temperature',
    xlim=(-5000, 85000),
    height=480,
    show_grid=True,
    multi_y=True
)

There are many ways for us to work with and visualize multivariate data, but it is highly dependent on the data. Data that consists of clusters of data can be visualized using methods like parallel coordinate/Andrews Curves; data that consists of categorical data may be representable with a radar plot; and sometimes the data just needs to be manipulated in ways to reduce what an analyst is looking at. We will soon be looking in more detail on how to cluster and group data, especially when working with data that is not directly categorical.

## Correlation & (Linear) Fitting

When working with more than one variable we can use various methods to quantify how well the variables correlate with one another. We meaure the correlation between two metrics using a scalar value inclusively between -1 and 1. This value is known as the *correlation coefficient*; values near +1 indicate a strong positive relation, values near -1 indicate a strong negative relation (inversely correlated), and values near or equal to 0 indicate weak or no relation.

### Pearson's Correlation Coefficient

This is the most commonly used correlation coefficient. It is simple to compute, and for us, as users of `numpy`, it is even easier (it is literally just a function call!).

$$
r = \frac{
        n\Sigma{xy}-(\Sigma{x})(\Sigma{y})
    }{
        [\sqrt{n\Sigma{x^{2}}-(\Sigma{x})^{2}][n\Sigma{y^{2}}-(\Sigma{y})^{2}}]
    }
$$

where `r` is our correlation coefficient, `x` and `y` are our data, and `n` is the sample size of the data. It does not *look* too simple at first glance, but it is and is pretty easy to compute. Consider the following dataset showing absences vs final grades for a set of students.

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'absences': [0, 1, 1, 2, 3, 3, 4, 5, 6, 7],
    'grade': [90, 85, 88, 84, 82, 80, 75, 60, 72, 64]
})
df

In [ ]:
df.hvplot.scatter(x='absences', y='grade', grid=True, title='Absences vs. Grade')

A quick reminder that we can reach into a data frame and grab particular columns. These columns are ultimately just `numpy` arrays, and so we can perform arithmetic operations quite easily on them! We are going create dummy variables `x`, `y`, and `n` to make write the code a bit easier.

In [ ]:
n = len(df)
x = df.absences
y = df.grade
n, x, y

Let's combine everything together and implement the equation for our correlation coefficient:

In [ ]:
import numpy as np
r = (n * (x * y).sum() - x.sum() * y.sum()) / np.sqrt((n * (x**2).sum() - x.sum()**2) * ((n*(y**2).sum() - y.sum()**2)))
r

The value of `-0.906` implies a strong negative correlation between the two variables (not necessarily causation, though in thise case there is grounds to believe causation). This makes sense - as a student misses school, their grades will likely suffer.

But boo! Why should we need to manually compute this value with all of that code above, especially one that is allegedly so common and popular? Well, luckily `numpy` gives us a pretty easy way to compute this particular metric.

In [ ]:
r = np.corrcoef(df.absences, df.grade)
r

The function [`numpy.corrcoeff`](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html) computes the correlation coefficients, and returns to us the *correlation matrix*, which shows the correlation between each variable against every variable, including itself. Each row of this matrix represents the coefficients each variable - the first row is how well it correlates to the set; the second row is how it correlates to the set; etc.The diagonals are 1 since any variable will be strongly correlated to itself. As this can be applied to more than just 2 variables at a time, the generalized form for computing this with `numpy` is:

In [ ]:
r = np.corrcoef([df.absences, df.grade])
r

Note the addition of the square braces. This denotes a *list* of metrics we want to compute the correlation coefficients for, and thus we are not limited to passing just two variables.

There are other means of computing correlation, but for most purposes this is sufficient. Some reminders and things to note about Pearson's Correlation:
 
* this does not indicate causation
* we cannot determine independent/dependent variables
* applicable only to linear relationships
* can be misleading in small sample sizes
* can be skewed due to clusters of data and outliers


## Linear Fitting

Another way we can assess correlation, and as discussed in reference to *Anscombe's Quartet*, is to produce and visualize linear trend lines overtop a scatter of the data. As we have briefly seen before, `numpy` also makes this easy. We are going to make use of `numpy`'s *Polynomial API*.

`numpy` implements a very powerful and flexible [Polynomial API](https://numpy.org/doc/stable/reference/routines.polynomials.html) with many polynomial finding routines implemented. Using a spread of data points we can determine approximate polynomial fittings. For our purposes for looking at correlation, we really only need to use a least squares fit.

In [ ]:
import holoviews as hv
import pandas as pd
from numpy.polynomial.polynomial import Polynomial
hv.extension('bokeh')

# load and prep data
data = pd.read_csv('data/anscombes_quartet.csv')
data.columns = pd.MultiIndex.from_product([['I', 'II', 'III', 'IV'], ['x', 'y']])

# create linear regression for first subset
fitted = Polynomial.fit(data['I'].x, data['I'].y, 1)
xs = np.array(data['I'].x)
ys = fitted(xs)

# scatter points and plot linear regression
scatter = hv.Scatter(data['I']).opts(size=5)
curve = hv.Curve({'x': xs, 'y':ys}).opts(color='red')
(scatter * curve).opts(height=480, width=700, show_grid=True, title="Anscombe's Quartet; Dataset I")

In [ ]:
fitted.coef

These coefficients are for a polynomial of the form:

$$
p(x) = c_{0} + c_{1}x + c_{2}x^{2} + ...
$$

### Where Linear Corrleation & Fitting Does Not Work

As mentioned, there are times when computing the correlation for a data set does not make sense. We are going to take a quick look at some examples of when we want to avoid looking at correlation and linear trends.

#### Nonlinear Data

As correlation is a linear statistic, it is not applicable to use for non-linear data. This does not mean the variables being assessed are not correlated or related in some way, just that we cannot quantify that relationship with the methods learned. Consider quadratic data and its trendline:

In [ ]:
# create non-linear data
quadratic = pd.DataFrame({
    'x': np.linspace(-5,5,25)
})
quadratic['y'] = quadratic.x**2

# create linear regresison
fitted = Polynomial.fit(quadratic.x, quadratic.y, 1)

# plot data with linear regression
scatter = hv.Scatter(quadratic).opts(size=5)
curve = hv.Curve({'x': quadratic.x, 'y':fitted(quadratic.x)}).opts(color='red')
(scatter * curve).opts(height=480, width=700, show_grid=True, title='Poor Linear Regression: Quadratic Data')


The trend line asserts that there is no variability within the data (slope == 0). Now let's compute Pearson's Correlation Coefficient:

In [ ]:
r = np.corrcoef(quadratic.x, quadratic.y)
r

Here we have a coefficient that is essentially 0, though there very well could be a relationship at work here (one that just is not linear!)

#### Small Sample Sizes

This one is applicable to more than just linear correlation - any sample size that is too small is hard to use to draw meaningful conclusions. Consider the following contrived dataset:

In [ ]:
# create small data
small = pd.DataFrame({
    'x': [1.0, 2.2, 3.8],
    'y': [0.5, 2.2, 2.3]
})

# create linear regresison
fitted = Polynomial.fit(small.x, small.y, 1)

# plot data with linear regression
scatter = hv.Scatter(small).opts(size=10)
curve = hv.Curve({'x': small.x, 'y':fitted(small.x)}).opts(color='red')
(scatter * curve).opts(height=480, width=700, show_grid=True, title='Poor Linear Regression: Small Data')

Now let's compute Pearson's Correlation Coefficient:

In [ ]:
r = np.corrcoef(small.x, small.y)
r

With our limited data we are told that there is a strong correlation between our variables, but 3 data points is hardly enough to really understand the data. A single additional data point that is within the range of our current data could greatly swing our correlation coefficient in either direction.

#### Clusters of Data

Our data may consist of various clusters of points - these clusters themselves may or may not have trends within, but collectively may show a trend altogether that may be misleading:

In [ ]:
# create clusters of data
clusters = pd.DataFrame({
    'x': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
    'y': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
})
# create linear regresison
fitted = Polynomial.fit(clusters.x, clusters.y, 1)

# plot data with linear regression
scatter = hv.Scatter(clusters).opts(size=5)
curve = hv.Curve({'x': clusters.x, 'y':fitted(clusters.x)}).opts(color='red')
(scatter * curve).opts(height=480, width=700, show_grid=True, title='Poor Linear Regression: Clustered Data')

In [ ]:
r = np.corrcoef(clusters.x, clusters.y)
r

There appears to be a very strong correlation here in our data, but it is likely more useful to evaluate the individual clusters. 

In [ ]:
# create clusters of data
clusters = pd.DataFrame({
    'x': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
    'y': np.concatenate([np.random.uniform(0.0, 0.5, 25), np.random.uniform(5.0, 5.5, 25)]),
})
# create linear regresison
fitted_1 = Polynomial.fit(clusters.iloc[:25].x, clusters.iloc[:25].y, 1)
fitted_2 = Polynomial.fit(clusters.iloc[25:].x, clusters.iloc[25:].y, 1)

scatter = hv.Scatter(clusters).opts(size=5)
curve_1 = hv.Curve({'x': clusters.iloc[:25].x, 'y':fitted_1(clusters.iloc[:25].x)}).opts(color='red')
curve_2 = hv.Curve({'x': clusters.iloc[25:].x, 'y':fitted_2(clusters.iloc[25:].x)}).opts(color='red')

(scatter * curve_1 * curve_2).opts(height=480, width=700, show_grid=True, title='Better Linear Regression: Individual Clusters')

In [ ]:
r = np.corrcoef(clusters.iloc[:25].x, clusters.iloc[:25].y)
r

In [ ]:
r = np.corrcoef(clusters.iloc[25:].x, clusters.iloc[25:].y)
r

#### Outliers

Outliers in data can greatly skew measures of correlation within a data set.

In [ ]:
# create clusters of data
outlier = pd.DataFrame({
    'x': np.random.uniform(0.0, 0.5, 25),
    'y': np.random.uniform(0.0, 0.5, 25),
})
outlier.iloc[-1] = [5.0, 5.0]

# create linear regresison
fitted = Polynomial.fit(outlier.x, outlier.y, 1)

# plot data with linear regression
scatter = hv.Scatter(outlier).opts(size=5)
curve = hv.Curve({'x': outlier.x, 'y':fitted(outlier.x)}).opts(color='red')
(scatter * curve).opts(height=480, width=700, show_grid=True, title='Poor Linear Regression: Outliers')

In [ ]:
r = np.corrcoef(outlier.x, outlier.y)
r

Again our trendline and correlation coefficient indicate a strong relation in the data, but the single outlier at `(5.0, 5.0)` is skewing our assessment of the data. If we strip that value from our data, we see that there is not much to our data.

In [ ]:
# remoec the outlier
outlier_removed = outlier.iloc[:-1]

# create linear regresison
fitted = Polynomial.fit(outlier_removed.x, outlier_removed.y, 1)

# plot data with linear regression
scatter = hv.Scatter(outlier_removed).opts(size=5)
curve = hv.Curve({'x': outlier_removed.x, 'y':fitted(outlier_removed.x)}).opts(color='red')
(scatter * curve).opts(height=480, width=700, show_grid=True, title='Better Linear Regression: Outlier Removed')

In [ ]:
r = np.corrcoef(outlier_removed.x, outlier_removed.y)
r

This is a much more accurate representation of our data! However, it is worth noting that outliers in data are not necessarily bad. There very well could be something in your model or system that behaves outlandish under certain criteria, and while it may make some statistical operations and observations worse/less conclusive, it is always best to understand the reasons behind any and all outliers!

## Exercise

Below we have some noisy data artificially representing radar measurements of a cannonball. Create a second-degree polynomial fit of the data to estimate the true path of the cannonball!

In [ ]:
# simulate noisy data
x = np.linspace(0, 10.0, 100)
y = 49.05*x - 4.905 * x * x + np.random.normal(0.0, 5.0, 100)
measurements = pd.DataFrame({'x':x, 'y':y})

# plot noisy data
scatter = hv.Scatter(measurements).opts(size=5)
scatter.opts(show_grid=True, height=480, width=700)

In [48]:
# implement and visualize the fit here!
